# Electronic structure through (quantum) annealing

In this project we map the electronic structure Hamiltonian to an Ising Hamiltonian and find the ground state energy.  Refer to the following references:

[1] https://arxiv.org/abs/1706.00271

[2] https://arxiv.org/abs/1811.05256

[3] https://arxiv.org/abs/1208.5986

We use molecular Hydrogen $H_2$ as an example. Assuming the atomic nucleus does not move due to its larger mass, the Hamiltonian which governs the electronic state can be transformed to a qubit representation appropriate for simulation on a quantum computer [3].  See Ref. [2], Eq. (6) for the $n$ qubit Hamiltonian, which encodes the electronic structure problem. Following Ref. [1], we then encode this problem in a classical Ising model, appropriate for annealing. This requires $r$ ancillary bit for each $n$ qubit.

The qubit Hamiltonian for moledular hydrogen $H_2$ is given by Eq. (37) in Ref. [1].  After the mapping described above, the problem eventually maps to the 2-local Ising-type Hamiltonian Eq. (41).  This goal becomes the calculation of the ground state energy of this Hamiltonian.



In [71]:
import numpy as np
from scipy.sparse import coo_matrix, csr_matrix
from scipy.special import logsumexp
import pickle

import pandas as pd

In [147]:
def energy(spins, J, h):
    # J - 2D np.array (assumed upper triangular)
    # h - 1D np.array
    # spins - 1D np.array (entries +/- 1)
    interaction = spins.dot(J.dot(spins))
    field = spins.dot(h)
    return interaction + field

def energy_diff(i, spins, J, h):
    return -4 * J[i, :].dot(spins) * spins[..., i] - 2 * h[i] * spins[..., i]

In [91]:
num_spins = 10


In [82]:
# random interaction+field ising model
J = np.random.randn(num_spins, num_spins)
J = np.triu(J, k=1)

h = np.random.randn(num_spins)
spins = (2*np.random.randint(2, size=(num_spins,)) - 1)

In [77]:
# standard classical ising with no field
J = np.zeros((num_spins, num_spins))
for i in range(J.shape[0]):
    J[i, (i+1) % num_spins] = -1

h = np.zeros(num_spins)
spins = (2*np.random.randint(2, size=(num_spins,)) - 1)

In [156]:
with open("./2localHamiltonians/OTIsHam5-2K.pkl", "rb") as f:
    h_, J_ = pickle.load(f, encoding="bytes")
    
num_spins = max(h_.keys()) + 1

h = np.zeros(num_spins)

for k, v in h_.items():
    h[k] = v

J = np.zeros((num_spins, num_spins))

for k, v in J_.items():
    J[k] = v
    
J = csr_matrix(J)

spins = (2*np.random.randint(2, size=(num_spins,)) - 1)

In [157]:
spins.shape

(126,)

In [158]:
def mc_step(spins, J, h, T):
    current_energy = energy(spins, J, h)
    for _ in range(spins.shape[0]):
        i = np.random.randint(spins.shape[0])        
        dE = energy_diff(i, spins, J, h)
        
        if (dE < 0) or (np.random.rand() < np.exp(-dE / T)):
            current_energy += dE
            spins[i] *= -1
        
    return spins, current_energy

In [159]:
T0 = T = 10.0
burn_in = 100
num_samples = 10000

for t in range(burn_in):
    mc_step(spins, J, h, T)
    

annealing_time = 10*burn_in
for t in range(annealing_time):
    T = T0 * np.exp(-t / annealing_time)
    mc_step(spins, J, h, T)
    
print(T)

3.682475046136629


In [160]:
print(T)
    
E = np.zeros(num_samples)
M = np.zeros(num_samples)

for t in range(num_samples):
    _, e = mc_step(spins, J, h, T)
    E[t] = e
    M[t] = np.abs(np.mean(spins))
    
(np.mean(E), np.std(E)/np.sqrt(num_samples)), (np.mean(M), np.std(M)/np.sqrt(num_samples))

3.682475046136629


((-116.96191433324269, 0.2941832156860005),
 (0.09612857142857142, 0.0005990870855767233))

In [139]:
size = num_spins
dim = np.arange(2 ** size)
space = ((dim[:, None] & (1 << np.arange(size))) > 0)[:, ::-1]
space = 2*space.astype(int) - 1

In [140]:
E = energy(space, J, h)
M = np.abs(np.mean(space, axis=-1))

logZ = logsumexp(-E / T)
probs = np.exp(-E / T - logZ)

np.dot(E, probs), np.dot(M, probs)

(-19.568758476047414, 0.1397789599433208)

In [132]:
min(E)

-19.750499361199676

In [99]:
0.99**1000

4.317124741065786e-05

Task 1.

1D Ising (2-local)
1D Ising (4-local)

Task 2.

Intro to spin glasses

1D Mattis Ising (2-local)
1D Mattis Ising (4-local)

Task 3. 

1D Random Bond Ising? (2/4-local)

Random Bond meaning J_ij still (next) nearest neighbour, but randomly positive or negative

Task 4.

H2 Annealing 

Might merge Tasks 1 and 2 together.